# Visualization Demo

We will cover the following examples on *new* demo images: 
- Visualizing transformed images
- Visualizing dilated convolution weights (learned kernels)
- Visualizing feature maps produced by dilated/transposed convolutions
- Example prediction on new (non-test set) demo images

In [57]:
import torch
import wandb
import os
import csv
import numpy as np
import pandas as pd
from models import XRayLightning
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
master_path = r'C:\Users\ericz\Documents\Github\APS360\Final Project'

In [58]:
run = wandb.init(project="lung-xray", entity="ericzhu",)
artifact = run.use_artifact(
    'ericzhu/lung-xray/model-q41lvdyh:v7', type='model')
artifact_dir = artifact.download()
model_checkpoint = os.path.join(artifact_dir, "model.ckpt")

wandb: Downloading large artifact model-q41lvdyh:v7, 744.89MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.9


In [66]:
# initialize model and load checkpoint
test_configs = {
    "master_path": master_path,
    "batch_size": 3000,
    "num_workers": 0,
    "max_epochs": 120,
    "lr": 0.00006,
    "weight_decay": 8e-8,
    "momentum": 0.98,
    "gamma": 2, 
    "use_inverse_weighting": False,
    "num_classes": 14,
    "fine_tune": True,
    "fine_tune_epoch_start": 40,
    "pos_weight_multi": 1.3,
    "train_from_scratch": False,
    "load_first": False,
    "checkpoint_path": model_checkpoint,
    "testing": True,
}
trunk_configs = {
    "trunk_input_channels": 1024,
    "trunk_mid_channels": 128,
    "trunk_out_channels": 64,
    "trunk_kernel_size": 7,
    "trunk_transpose_kernel": 21,
    "trunk_dropout": 0.15,
    "trunk_conv_layers": 2,
}
head_configs = {
    "use_vit": True,
    "head_n_layer": 52,
    "head_n_head": 4,
    "head_feature_map_dim": 16,
    "head_input_channels": 64,
    "head_mid_channels": 32,
    "head_output_channels": 16,
    "head_kernel_size": 4,
    "head_max_pool_kernel_size": 2,
    "head_conv_layers": 1,
    "head_classifier_input_features": 512,
    "head_hidden_size": 384,
    "head_dropout": 0.5,
}
# combine configs into train_configs
test_configs.update(trunk_configs)
test_configs.update(head_configs)

# initalize model with configs
lightning_model = XRayLightning(**test_configs)

# load checkpoint
lightning_model = XRayLightning.load_from_checkpoint(model_checkpoint)
lightning_model.eval()
lightning_model.freeze()
model_predictor = lightning_model.model.cuda()

# print model atrributes
print(lightning_model.hparams)
print(lightning_model)

14 True


c:\Users\ericz\anaconda3\envs\pytorch-latest\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ericz\anaconda3\envs\pytorch-latest\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


14 True
"alpha":                          None
"batch_size":                     128
"data_size":                      70960
"fine_tune":                      True
"fine_tune_epoch_start":          40
"gamma":                          2
"head_classifier_input_features": 512
"head_conv_layers":               1
"head_dropout":                   0.5
"head_feature_map_dim":           16
"head_hidden_size":               384
"head_input_channels":            64
"head_kernel_size":               4
"head_max_pool_kernel_size":      2
"head_mid_channels":              32
"head_n_head":                    4
"head_n_layer":                   52
"head_output_channels":           16
"lr":                             6e-05
"momentum":                       0.98
"num_classes":                    14
"num_workers":                    0
"pos_weight_vec":                 tensor([  8.4792,  40.5272,  23.4376,  49.1898,   7.1421,  44.8858,  67.7948,
        514.1385,   4.2354,  18.5997,  16.8438,  32.9391

In [4]:
# load test predictions CSV 
pred_csv_path = os.path.join(master_path, "lung_detection", "test_predictions_final.csv")

#test_predictions = pd.read_csv(pred_csv_path, converters={"is_correct": ast.literal_eval, "predictions": ast.literal_eval, "label": ast.literal_eval, "probs": ast.literal_eval, "img_path": ast.literal_eval})

test_predictions = pd.read_csv(pred_csv_path)

# filter for correct predictions
correct_predictions = test_predictions[test_predictions["is_correct"] == "[ True]"]

# remove the brackets from the is_correct column
correct_predictions["is_correct"] = correct_predictions["is_correct"].str.replace("[", "")
correct_predictions["is_correct"] = correct_predictions["is_correct"].str.replace("]", "")

# remove brackets from img_path column
correct_predictions["img_path"] = correct_predictions["img_path"].str.replace("[", "")
correct_predictions["img_path"] = correct_predictions["img_path"].str.replace("]", "")

# remove quotes from img_path column
correct_predictions["img_path"] = correct_predictions["img_path"].str.replace("'", "")


correct_predictions

In [ ]:
# select 10 random correct predictions
random_correct_predictions = correct_predictions.sample(n=10)

In [84]:
# create a list of the image paths from path 
new_data_path = os.path.join(master_path, "lung_detection", "NewData")
new_disease_path = os.path.join(new_data_path, "Pneumothorax")

# get a list of files in this directory
new_image_files = [os.path.join(new_disease_path, file_name) for file_name in os.listdir(new_disease_path)]

In [85]:
run_demo = True
# run the model on the 10 random correct predictions
img_paths = random_correct_predictions["img_path"].tolist() if not run_demo else new_image_files
mean = torch.tensor([0.485, 0.456, 0.406])
std = torch.tensor([0.229, 0.224, 0.225])

In [131]:
# further transform the images to be fed into the model
transform = transforms.Compose([
    transforms.Lambda(lambda x: x.convert('RGB')),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.AugMix(severity=6, mixture_width=6),
    transforms.TrivialAugmentWide(),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

for img_path in img_paths:
    # read img_path as raw string
    img_path = fr"{img_path}"
    with Image.open(img_path) as img:
        img.load()
    img = transform(img)
    img = img.permute(1, 2, 0)
    img = img.numpy()
    img = np.clip(img, 0, 1)
    img = Image.fromarray((img * 255).astype(np.uint8))
    img.save(os.path.join(master_path, "lung_detection", "transformed_train_examples", img_path.split("\\")[-1]))


In [132]:
# show the learned kernel as an image
# save the image to "kernel weights visualized" folder
# file names should be "kernel weights visualized/img_path_validation.png"
# use the same transforms as above

# get the kernel weights
kernel_weights = lightning_model.model.trunk.trunk[1].block[0].weight.data.cpu().numpy()
print(kernel_weights.shape)

# save the kernel weights as images
for i in range(kernel_weights.shape[0]):
    kernel = kernel_weights[i, 0, :, :]
    kernel = kernel - kernel.min()
    kernel = kernel / (kernel.max() - kernel.min())
    kernel = kernel * 255
    kernel = kernel.astype(np.uint8)
    kernel = Image.fromarray(kernel)
    kernel.save(os.path.join(master_path, "lung_detection", "transposed_kernel_weights_visualized", f"kernel_{i}.png"))


(64, 64, 21, 21)


In [133]:
# get the kernel weights
kernel_weights = lightning_model.model.trunk.trunk[3].blocks[0].block[0].weight.data.cpu().numpy()
print(kernel_weights.shape)

# save the kernel weights as images
for i in range(kernel_weights.shape[0]):
    kernel = kernel_weights[i, 0, :, :]
    kernel = kernel - kernel.min()
    kernel = kernel / (kernel.max() - kernel.min())
    kernel = kernel * 255
    kernel = kernel.astype(np.uint8)
    kernel = Image.fromarray(kernel)
    kernel.save(os.path.join(master_path, "lung_detection", "dilated_kernel_weights_visualized", f"kernel_{i}.png"))


(128, 64, 7, 7)


In [134]:
# run the model on the 10 random correct predictions
# do so on the dense_model and trunk modules of the model
# save to the "embeddings_visualized" folder
transform = transforms.Compose([
    transforms.Lambda(lambda x: x.convert('RGB')),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
     transforms.Normalize(mean=mean, std=std)
])

for img_path in img_paths:
    img_path = fr"{img_path}"
    img = Image.open(img_path)
    img = transform(img)
    img = img.unsqueeze(0)
    img = img.cuda()
    with torch.no_grad():
        dense_model_output = model_predictor.dense_model(img)
        trunk_output = model_predictor.trunk(dense_model_output)
        trunk_output = trunk_output*255
    trunk_output = torch.nn.functional.interpolate(trunk_output, size=(128, 128), mode="bilinear")
    num_channels = trunk_output.shape[1]
    trunk_output = trunk_output.cpu().numpy()

    # loop over channels, and save each channel as a separate image
    for i in range(num_channels):
        channel = trunk_output[0, i, :, :]
        channel = np.clip(channel, 0, 255)
        channel = Image.fromarray(channel.astype(np.uint8))
        channel.save(os.path.join(master_path, "lung_detection", "embeddings_visualized", img_path.split("\\")[-1].split(".")[0] + f"_channel_{i}.png"))

In [135]:
# predict on the img_paths and print out the multi-label predictions
transform = transforms.Compose([
    transforms.Lambda(lambda x: x.convert('RGB')),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
     transforms.Normalize(mean=mean, std=std)
])
for img_path in img_paths: 
    img_path = fr"{img_path}"
    img = Image.open(img_path)
    img = transform(img)
    img = img.unsqueeze(0)
    img = img.cuda()
    with torch.no_grad():
        prediction = lightning_model(img)
        prediction = torch.sigmoid(prediction)
        prediction = prediction.cpu().numpy()
        prediction = prediction[0]
        prediction = prediction > 0.5
        print(prediction)

[ True  True  True  True  True False False False  True  True  True False
 False False]
[ True  True  True  True  True False False False  True  True  True False
 False False]
[ True  True  True  True  True  True  True False  True  True  True  True
 False  True]
